In [1]:
import sys
sys.path.insert(0, '../python')
%load_ext autoreload
%autoreload 2

# 测试 ConvLSTM

In [2]:
from ConvLSTM import *
import torch

In [3]:
batch_size, input_channel, hidden_size, width, heigt = 2, 10, 10, 64, 32
c = ConvLSTMCell(input_channel, hidden_size, 3)

In [4]:
loss_fn = torch.nn.MSELoss()

In [5]:
input = torch.randn(batch_size, input_channel, heigt, width)
target = torch.randn(batch_size, input_channel, heigt, width)
hidden = torch.zeros(batch_size, hidden_size, heigt, width)
cell = torch.zeros(batch_size, hidden_size, heigt, width)

In [6]:
output = c(input, (hidden, cell))[0]

In [10]:
if torch.cuda.is_available():
    output = output.cuda()
    target = target.cuda()
output = output.double()
target = target.double()
torch.autograd.gradcheck(loss_fn, (output, target), eps=1e-6, raise_exception=True)

True

In [14]:
seq_num = 10
input = torch.randn(batch_size, seq_num, input_channel, heigt, width)
target = torch.randn(batch_size, seq_num, input_channel, heigt, width)

state = (hidden, cell)

In [15]:
conv_lstm = ConvLSTM(input_channel, hidden_size, 3)

In [18]:
output, stack = conv_lstm(input, state)

In [20]:
if torch.cuda.is_available():
    output = output.cuda()
    target = target.cuda()
output = output.double()
target = target.double()
torch.autograd.gradcheck(loss_fn, (output, target), eps=1e-6, raise_exception=True)

True

# 测试 Encoder

In [47]:
from model import *

In [48]:
encoder = Encoder(ConvLSTM)

In [49]:
encoder

Encoder(
  (stage1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
  )
  (rnn1): ConvLSTM(
    (_cell): ConvLSTMCell(
      (_conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage2): Sequential(
    (0): Conv2d(64, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (rnn2): ConvLSTM(
    (_cell): ConvLSTMCell(
      (_conv): Conv2d(384, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage3): Sequential(
    (0): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (rnn3): ConvLSTM(
    (_cell): ConvLSTMCell(
      (_conv): Conv2d(384, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
)

In [50]:
data = torch.randn(2, 5, 1, 501, 501)

In [52]:
state = encoder(data)

torch.Size([2, 5, 64, 84, 84])
torch.Size([2, 5, 192, 42, 42])
torch.Size([2, 5, 192, 21, 21])
